# Тензоры

## От списков к тензорам

In [1]:
a = [1.0, 2.0, 3.0]

In [2]:
a[0]

1.0

In [3]:
a[2]

3.0

In [4]:
a

[1.0, 2.0, 3.0]

## Первые тензоры!

In [5]:
import torch

In [6]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [7]:
a[1]

tensor(1.)

In [8]:
float(a[1])

1.0

In [9]:
a[2] = 2.0

In [10]:
a

tensor([1., 1., 2.])

### Тензор координат

In [11]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [12]:
points[0] = torch.tensor([1, 2])

In [13]:
points

tensor([[1., 2.],
        [0., 0.],
        [0., 0.]])

In [14]:
points[0, 1]

tensor(2.)

In [15]:
points[0][1]

tensor(2.)

In [16]:
# Можно и так, и так обращаться к элементам

In [17]:
points[None]

tensor([[[1., 2.],
         [0., 0.],
         [0., 0.]]])

In [18]:
points.unsqueeze(0)

tensor([[[1., 2.],
         [0., 0.],
         [0., 0.]]])

In [19]:
points

tensor([[1., 2.],
        [0., 0.],
        [0., 0.]])

In [20]:
# unsqueeze увеличивает размерность тензора на 1. [None] делает тоже самое в действительности

### Тензоры поименованные

In [21]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([.2126, .7152, .0722])

In [22]:
batch_t = torch.randn(2, 3, 5, 5)

In [23]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [24]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)

### Именованные тензоры

In [25]:
weights_named = torch.tensor([.2126, .7152, .0722], names=['channels'])
weights_named

/tmp/ipykernel_5996/230612884.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1675740247391/work/c10/core/TensorImpl.h:1758.)
  weights_named = torch.tensor([.2126, .7152, .0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [26]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named:', img_named.shape, img_named.names)
print('batch named:', batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [28]:
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [36]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned

tensor([[[0.2126]],

        [[0.7152]],

        [[0.0722]]], names=('channels', 'rows', 'columns'))

In [37]:
weights_named.shape, weights_aligned.shape

(torch.Size([3]), torch.Size([3, 1, 1]))

In [38]:
gray_named = (img_named * weights_aligned).sum('channels')

### Работа с dtype у тензоров

In [42]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [43]:
short_points.dtype

torch.int16

In [44]:
double_points.dtype

torch.float64

In [45]:
# Можно привести к другому виду несколькими способами

In [46]:
double_points = torch.zeros(10, 3).double()
short_points = torch.ones(5, 5).short()


double_points = torch.zeros(10, 3).to(torch.double)
short_points = torch.ones(5, 5).to(dtype=torch.short)

In [47]:
points_64 = torch.randn(5, dtype=torch.double)
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0.0000, -0.0000, 1.5098, 5.6320, 1.1804], dtype=torch.float64)

# API тензоров

In [71]:
a = torch.ones(3, 2)
a_t = a.T

In [72]:
a

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

In [73]:
a_t

tensor([[1., 1., 1.],
        [1., 1., 1.]])

## Сериализация тензоров

In [74]:
import h5py

In [75]:
f = h5py.File('ourpoints.hdf5', 'w')

dset = f.create_dataset('coords', data=points.numpy())

In [83]:
f = h5py.File('./ourpoints.hdf5', 'r')
dset = f['coords']
last_points = dset[:]
f.close()
print(last_points)

[[1. 2.]
 [0. 0.]
 [0. 0.]]
